# Creating the Tools with Standalone Callbacks 
Goal: create the tools so that people can use them as a python script

Steps to complete the main tool:
1. set up changeable channels with the most basic barebone example
2. link the two plots together
3. link to the frequency slider (mind the javacall back: callback_policy="mouseup")
4. set up histogram and bin changer
5. link histogram data. make sure it can be updated
6. fix minor details such as plot labels etc.

In [1]:
import sys
# locate your spectralCV so we have scv_funcs to use
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/notebooks_visualization/')

sys.path.append('/Users/rdgao/Documents/code/research/spectralCA/')
sys.path.append('/Users/rdgao/Documents/code/research/spectralCA/notebooks_visualization/')

In [2]:
import os
# the only lines you need to change to inspect different saved data
# path = '/Users/ldliao/Research/Projects/spectralCA/results/kjm_digits/bp/'
# path = '/Users/ldliao/Research/Projects/spectralCA/results/nt/'
path = '/Users/ldliao/Research/Projects/spectralCA/results/dino/'

# path = '/Users/rdgao/Documents/code/research/spectralCA/results/dino/'

os.chdir(path) # DONT CHANGE PATH INSIDE THE NOTEBOOK, YOU"LL GET LOST. USE FULL PATH
import glob
from sca_funcs import sca

In [3]:
# the final product to import the visualization
import sca_vis

Loading BokehJS ...

In [4]:
# sc = sca.sca_load_spec('nt_sca2_Anes.npz')
#sc = sca.sca_load_spec('sleep.npz')
sc = sca.sca_load_spec('wake.npz')

In [5]:
sca_vis.plot_pct_spectrogram(sc=sc)

In [6]:
# ignore bokeh warnings
import warnings
warnings.filterwarnings('ignore')

sca_vis.plot_vis(sc=sc)

### Imports

In [ ]:
# imports
import numpy as np
import scipy as sp
from scipy.stats import expon
import glob

# import neurodsp as ndsp
# from scv_funcs import lfpca
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# bokeh imports
import bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Slider, Span
from bokeh.models.widgets import Select, Slider
from ipywidgets import Layout, HBox, interactive

output_notebook()

### Defaults

In [ ]:
# setting up the defaults
chan = 1
select_freq = 10
select_bin = 20

freq_vals = sc.f_axis[1:]
psd_vals = sc.psd[chan].T[1:]
scv_vals = sc.scv[chan].T[1:]
real_vals = sc.spg.real[chan][select_freq]
imag_vals = sc.spg.imag[chan][select_freq]

source = ColumnDataSource(data=dict(freq_vals=freq_vals, 
                                    psd_vals=psd_vals, 
                                    scv_vals=scv_vals))

complex_source = ColumnDataSource(data=dict(real_vals=real_vals, imag_vals=imag_vals))
# grabbing channel count from psd
chan_count, freq = sc.psd.shape
DEFAULT_TICKERS = list(map(str, range(chan_count)))

In [ ]:
# # set up complex plot
# complex_plot = figure(title='Complex SPG', plot_width=300, plot_height=300)
# complex_plot.legend.location = 'top_left'
# complex_plot.xaxis.axis_label = 'Real values'
# complex_plot.yaxis.axis_label = 'Imaginary values'
# complex_plot.grid.grid_line_alpha=0.3
# complex_plot.circle('real_vals', 'imag_vals', size=5, color="purple", alpha=0.5, source=complex_source)

# # create interact
# def update_spct(channel=1, f=10):    
#     plot_chan = int(channel)
#     plot_freq = np.where(sc.f_axis==f)[0][0]

#     # update complex data
#     complex_source.data['real_vals'] = sc.spg.real[plot_chan][plot_freq]
#     complex_source.data['imag_vals'] = sc.spg.imag[plot_chan][plot_freq]
    
#     push_notebook()
    
# show(complex_plot, notebook_handle=True)
    
# warnings.filterwarnings('ignore')
# widget = interactive(update_spct, channel=range(1,len(DEFAULT_TICKERS)-1), f=(1,199), numbins=(10,55,5))
# items = [kid for kid in widget.children]
# display(HBox(children=items))

### Sliders and updates

In [ ]:
# create interact
def update_spct(channel=1, f=10, numbins=20):    
    plot_chan = int(channel-1)
    plot_freq = np.where(sc.f_axis==f)[0][0]
    y, x = np.histogram(abs(sc.spg[plot_chan,plot_freq,:])**2, bins=numbins, density=True)

    # create a column data source for the plots to share
    data_source = {
                   'freq_vals': sc.f_axis[1:],
                   'psd_vals': sc.psd[plot_chan].T[1:],
                   'scv_vals': sc.scv[plot_chan].T[1:]
                   }   
    source.data = data_source
    vline_psd.location = f
    vline_scv.location = f
    # update histogram with data from frequency f
    hist_plot.data_source.data['left'] = x[:-1]
    hist_plot.data_source.data['right'] = x[1:]
    hist_plot.data_source.data['top'] = y    
    # update fitted
    rv = expon(scale=sp.stats.expon.fit(abs(sc.spg[plot_chan,plot_freq,:])**2,floc=0)[1])
    fit_plot.data_source.data['x'] = x
    fit_plot.data_source.data['y'] = rv.pdf(x)

    # update complex data
    complex_source.data['real_vals'] = sc.spg.real[plot_chan][f]
    complex_source.data['imag_vals'] = sc.spg.imag[plot_chan][f]

    hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(f, sc.ks_pvals[int(plot_chan), f])
    push_notebook()

### Set up the plots

In [ ]:
plot_side_length = 310
    
# set up histogram
y, x = np.histogram(abs(sc.spg**2)[0,10,:], bins=20, density=True)
hist_fig = figure(plot_height=plot_side_length, plot_width=plot_side_length, x_axis_label='Power', y_axis_label='Probability')
hist_plot = hist_fig.quad(top=y, bottom=0, left=x[:-1], right=x[1:], fill_color='purple', line_color="lightblue")
hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(10, sc.ks_pvals[int(1), 10])
hist_fig.axis.major_label_text_font_size= '0pt'
fit_plot = hist_fig.line(x[:-1],y, line_width=8,alpha=0.7,line_color="#D53B54",legend='Fit PDF')

# set up psd plot
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log', plot_width=plot_side_length, plot_height=plot_side_length)
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3
psd_plot.line('freq_vals', 'psd_vals', source=source)    

# set up scv plot
scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log', plot_width=plot_side_length, plot_height=plot_side_length)
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3
fit_line = bokeh.models.glyphs.Line(x='freq_vals', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
scv_plot.add_glyph(source, fit_line)
scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')

# set up complex plot
complex_plot = figure(title='Complex SPG', plot_width=plot_side_length, plot_height=plot_side_length)
complex_plot.legend.location = 'top_left'
complex_plot.xaxis.axis_label = 'Real values'
complex_plot.yaxis.axis_label = 'Imaginary values'
basic_xline = bokeh.models.glyphs.Line(x='real_vals', y=0, line_width=5, line_alpha=0.5, line_color='darkgrey')
complex_plot.add_glyph(complex_source, basic_xline)
basic_yline = bokeh.models.glyphs.Line(x=0, y='imag_vals', line_width=5, line_alpha=0.5, line_color='darkgrey')
complex_plot.add_glyph(complex_source, basic_yline)
complex_plot.grid.grid_line_alpha=0.3
complex_plot.circle('real_vals', 'imag_vals', size=5, color="purple", alpha=0.5, source=complex_source)


# add in frequency slider vertical lines
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

# set up layout and interact toola
layout = column(row(psd_plot, scv_plot), row(hist_fig, complex_plot))
show(layout, notebook_handle=True)

warnings.filterwarnings('ignore')
widget = interactive(update_spct, channel=range(1,len(DEFAULT_TICKERS)+1), f=(1,199), numbins=(10,55,5))
items = [kid for kid in widget.children]
display(HBox(children=items))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets

from bokeh.palettes import viridis
from bokeh.models.glyphs import Rect
from bokeh.models import BoxAnnotation

In [ ]:
def percentile_spectrogram(spg, f_axis, rank_freqs=(8., 12.), pct=(0, 25, 50, 75), sum_log_power=True, show=True):
    f_ind = np.where(np.logical_and(
        f_axis >= rank_freqs[0], f_axis <= rank_freqs[1]))

    if sum_log_power:
        power_vals = np.sum(np.log10(spg[f_ind, :][0]), axis=0)
    else:
        power_vals = np.sum(spg[f_ind, :][0], axis=0)

    bins = np.percentile(power_vals, q=pct)
    power_dgt = np.digitize(power_vals, bins, right=False)
    power_binned = np.asarray(
        [np.mean(spg[:, power_dgt == i], axis=1) for i in np.unique(power_dgt)])
    
    return power_dgt, power_binned

In [ ]:
# setting up the defaults
chan = 1 #10,100
rank_freqs = (10,20)
pct = 20
plot_side_length = 350

# calculating the percentile spectrogram
pct_range = range(0,100,pct)
power_dgt, power_binned = percentile_spectrogram(np.abs(sc.spg[chan,:,:]**2), sc.f_axis, rank_freqs, pct_range);
numlines = power_binned.T[1:].shape[1]

# setting into data format
freq_vals = [sc.f_axis[1:]]*numlines
power = list(zip(*power_binned.T[1:].tolist()))
source = ColumnDataSource(data=dict(freq_vals=freq_vals, 
                                    power=power,
                                    color=palette))
# color it accordingly
palette = viridis(numlines)

# set up plot
pct_spct_plot = figure(title='Percentile Spectrogram', x_axis_type='log', y_axis_type='log', plot_width=plot_side_length, plot_height=plot_side_length)
pct_spct_plot.legend.location = 'top_left'
# adding the box of grey
bg_box = BoxAnnotation(left=rank_freqs[0], right=rank_freqs[1], fill_color='grey', fill_alpha=0.2, line_alpha=0)
pct_spct_plot.add_layout(bg_box)
# plotting the lines
pct_spct_plot.multi_line(xs='freq_vals', ys='power', color='color', source=source)

show(pct_spct_plot, notebook_handle=True)

In [ ]:
# create interact
def update_pct_spct(channel=1, rank_freqs=(10,20), pct=range(0,100,20)): 
    
    # updating information based on sliders
    plot_chan = int(channel-1)
    pct_range = range(0,100,pct)
    power_dgt, power_binned = percentile_spectrogram(np.abs(sc.spg[plot_chan,:,:])**2, sc.f_axis, rank_freqs, pct_range);
    numlines = power_binned.T[1:].shape[1]
    freq_vals = [sc.f_axis[1:]]*numlines
    power = list(zip(*power_binned.T[1:].tolist()))

    palette = viridis(numlines)
    
    # create a column data source for the plots to share
    updated_source = dict(freq_vals=freq_vals, power=power, color=palette)

    source.data = updated_source
    
    # update the box
    bg_box.left = rank_freqs[0]
    bg_box.right = rank_freqs[1]
    push_notebook()

In [ ]:
chan_count, freq = sc.psd.shape

DEFAULT_TICKERS = list(map(str, range(chan_count)))

# defining the widge we are using
widget = interactive(update_pct_spct, channel=range(1,len(DEFAULT_TICKERS)+1), 
                                 rank_freqs=widgets.IntRangeSlider(
                                            value=[10, 20],
                                            min=0,
                                            max=100,
                                            step=1,
                                            description='rank_freqs',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',
                                        ), pct=(1,100,10))
items = [kid for kid in widget.children]
display(HBox(children=items))